In [7]:
# read csv and check the data 
import torch
from torch import nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os

In [8]:
# data washing
file_name = r"./housing.csv"
data = pd.read_csv(file_name)
# data.drop_duplicates(["ocean_proximity"])  observe the data set 

split_rate = 0.7 # the split partial of training set and testing set
train_path = "train_housing.csv" 
test_path = "test_housing.csv" 

In [12]:
# check is there any NA value
data.isna().sum()

def str2number(str):
    """convert the string value of ocean_proximity to number and train"""
    if str == "ISLAND":
        return 0
    elif str == "NEAR OCEAN":
        return 1
    elif str == "NEAR BAY":
        return 2
    elif str == "<1H OCEAN":
        return 3
    else:
        return 4

data["ocean_proximity"] = data["ocean_proximity"].map(str2number)
data["total_bedrooms"].interpolate("linear", inplace=True)

def load_data(file_name):
    data = pd.read_csv(file_name)
    data["ocean_proximity"] = data["ocean_proximity"].map(str2number)
    data["total_bedrooms"].interpolate("linear", inplace=True)
    return data


data.columns[0]
data.loc[:, "ocean_proximity"]

def save_data(data, file_name):
    data.to_csv(file_name, index=False)

def split_data(data, split_rate, path):
    "split the original dataframe and save to train_file_name file"
    tmp = int(len(data) * split_rate)
    train_data = data[0:tmp]
    test_data = data[tmp:]
    train_path, test_path = path
    save_data(train_data, train_path)
    save_data(test_data, test_path)

split_data(data, split_rate, (train_path, test_path))

In [14]:
class housedataset(Dataset):
    def __init__(self, filename):
        self.data = load_data(filename)
        self.column_length = len(self.data.columns)
        self.data_features = self.data.iloc[:,[x for x in range(self.column_length) if self.data.columns[x] != "median_house_value"]]
        self.targets = self.data.loc[:, "median_house_value"]
    
    def __getitem__(self, index):
        # how to extract the data and labels from the whole data

        data_features, targets = self.data_features.iloc[index], self.targets.iloc[index] # index first

        # print(data_features.values.shape)

        targets = targets if type(targets) != pd.Series else targets.values

        data = torch.tensor(data_features.values, dtype=torch.float32) # convert dataframe into tensor
        targets = torch.tensor(targets, dtype=torch.float32)

        return data, targets


    def __len__(self):
        return len(self.data)

In [47]:
# set hyper parameters
epochs = 1000
device = torch.device("cuda") if torch.cuda.is_available else torch.device("cpu")
lr = 0.01
batch_size = 200


In [48]:
# Load the data

# dataset = housedataset(filename) # use our own value to make dataset

train_set = housedataset(train_path)
test_set = housedataset(test_path)

# this operation is really wrong !!!!! The train_set is not dataset
# train_set = dataset[0:int(split_rate * len(dataset))]
# test_set = dataset[int(split_rate * len(dataset)) : ]

# batchsize = 64


train_data = DataLoader(train_set, batch_size, shuffle=True, drop_last=True)
test_data = DataLoader(test_set, batch_size, shuffle=True, drop_last=True)

# test the data loader
for data in train_data:
    values, targets = data
    print(values.shape)
    # print(targets)
    break


torch.Size([200, 9])


In [49]:
# define neural network
class Housing(nn.Module):
    def __init__(self):
        super(Housing, self).__init__()
        self.lin1 = nn.Linear(9, 20) 
        self.relu1 = nn.ReLU()
        self.lin2 =nn.Linear(20, 5)
        self.lin3 = nn.Linear(5, 1)

    def forward(self, x):
        x = self.lin1(x)
        x = self.relu1(x)
        x = self.lin2(x)
        x = self.relu1(x)
        x = self.lin3(x)
        return x

house = Housing()

# define loss function
loss_fn = nn.MSELoss().to(device)

optim = torch.optim.SGD(house.parameters(), lr=lr)

In [50]:
# training the model
for epoch in range(epochs):
    house.train() # start the training model
    for i, data in enumerate(train_data):
        values, targets = data
        values.to(device)
        targets.to(device)

        optim.zero_grad()
        res = house(values)
        targets = targets.reshape(res.shape)
        train_loss = loss_fn(res, targets)
        train_loss.backward()
        optim.step()

        if i % 100 == 0:
            print(train_loss)
            
    house.eval()
    with torch.no_grad():
        total_loss = torch.tensor(0.0)
        num = 0
        for data in test_data:
            values, targets = data
            values.to(device)
            targets.to(device)

            res = house(values)
            targets = targets.reshape(res.shape)
            train_loss = loss_fn(res, targets)

            total_loss += train_loss
            num += 1
        print("\n test: \n total_loss is {}, avg_loss is {} \n".format(total_loss, total_loss / num))
        



tensor(5.6060e+10, grad_fn=<MseLossBackward0>)

 test: 
 total_loss is inf, avg_loss is inf 

tensor(inf, grad_fn=<MseLossBackward0>)

 test: 
 total_loss is 5.737605920186806e+36, avg_loss is 1.9125353595477103e+35 

tensor(1.9125e+35, grad_fn=<MseLossBackward0>)

 test: 
 total_loss is 3.1281356319193497e+35, avg_loss is 1.0427118195359147e+34 

tensor(1.0427e+34, grad_fn=<MseLossBackward0>)

 test: 
 total_loss is 1.7054566369077752e+34, avg_loss is 5.684855482149668e+32 

tensor(5.6849e+32, grad_fn=<MseLossBackward0>)

 test: 
 total_loss is 9.29813513914646e+32, avg_loss is 3.099378492548563e+31 

tensor(3.0994e+31, grad_fn=<MseLossBackward0>)

 test: 
 total_loss is 5.06932674238806e+31, avg_loss is 1.6897756110191656e+30 

tensor(1.6898e+30, grad_fn=<MseLossBackward0>)

 test: 
 total_loss is 2.763790900446718e+30, avg_loss is 9.21263620889262e+28 

tensor(9.2126e+28, grad_fn=<MseLossBackward0>)

 test: 
 total_loss is 1.5068155307739361e+29, avg_loss is 5.02271843591312e+27 

t

KeyboardInterrupt: 